In [1]:
!pip install faster-whisper
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 58.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Server

In [2]:
from google.colab import userdata
NGROK_TOKEN = userdata.get('NGROK_TOKEN')
!ngrok config add-authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import StreamingResponse
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from faster_whisper import WhisperModel
from io import BytesIO
import time

app = FastAPI()

# Load the Whisper model
model_size = "Systran/faster-distil-whisper-medium.en"
model_size = "distil-large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="float16")

@app.post("/transcribe_stream")
async def transcribe_stream(audio: UploadFile = File(...), initial_prompt: str = Form("")):
    # Save the uploaded audio file to a BytesIO object
    audio_bytes = BytesIO(await audio.read())

    # Transcribe the audio file
    segments, info = model.transcribe(audio_bytes, beam_size=5, language="en", condition_on_previous_text=False, word_timestamps=True, initial_prompt=initial_prompt)

    # Stream the transcription segments
    async def stream_segments():
        for segment in segments:
            yield f"{segment.text}\n"

    return StreamingResponse(stream_segments(), media_type="text/plain")

@app.post("/transcribe")
async def transcribe(audio: UploadFile = File(...), initial_prompt: str = Form("")):
    # Save the uploaded audio file to a BytesIO object
    audio_bytes = BytesIO(await audio.read())

    # Transcribe the audio file
    start = time.time()
    segments, info = model.transcribe(audio_bytes, beam_size=5, language="en", condition_on_previous_text=False, word_timestamps=True, initial_prompt=initial_prompt)

    # Convert segments to a list
    segments = list(segments)
    end = time.time()
    print("Transcription time: %.2fs" % (end - start))

    json_segments = []

    for s in segments:
        sg = {}
        for field in ['id', 'seek', 'start', 'end', 'text', 'words']:
            sg[field] = getattr(s, field)
            sg['words'] = []
            for word in s.words:
                sg['words'].append({field: getattr(word, field) for field in ['start', 'end', 'word']})

        json_segments.append(sg)

    return {"segments": json_segments}

reserved_domain = "colab.ngrok.pro"
ngrok_tunnel = ngrok.connect(8000, hostname=reserved_domain)
print(f"Public URL: https://{reserved_domain}")
# ngrok_tunnel = ngrok.connect(8000)
# print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

uvicorn.run(app, host='0.0.0.0', port=8000)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

INFO:     Started server process [4036]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://colab.ngrok.pro
INFO:     2001:f40:910:1983:9904:f9e6:90cc:f800:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:9904:f9e6:90cc:f800:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:9904:f9e6:90cc:f800:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:9904:f9e6:90cc:f800:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:9904:f9e6:90cc:f800:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:9904:f9e6:90cc:f800:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
